# Imports

In [0]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Widgets and Parameters

In [0]:
dbutils.widgets.text("scenario", "all")
scenario_run = dbutils.widgets.get("scenario").lower()

In [0]:
scenario_run

'default rigs'

In [0]:
# condition preparation
if scenario_run == "all":
    scenario_condition = "and scenario_name = 'default'"
else:
    scenario_condition = f"and scenario_name = '{scenario_run}'"

In [0]:
# %sql
# insert into table produced.drillout_model_scenarios (scenario_name, basin, flow_unit, number_of_rigs, forecast_years, cycle_times_min_cutoff_years, rigs_min_cutoff_years, active_rigs_min_cutoff_months) values ("default rigs", "GULF COAST EAST", "HAYNESVILLE", null, 15, 5, 2, 6)

In [0]:
%sql
select * from produced.drillout_model_scenarios

scenario_id,scenario_name,basin,flow_unit,number_of_rigs,forecast_years,cycle_times_min_cutoff_years,rigs_min_cutoff_years,active_rigs_min_cutoff_months
2,testing,GULF COAST EAST,HAYNESVILLE,35,15,5,2,6
4,default rigs,GULF COAST EAST,HAYNESVILLE,null,15,5,2,6
1,default,GULF COAST EAST,HAYNESVILLE,30,15,5,2,6
3,testing,GULF COAST EAST,HAYNESVILLE,null,15,5,2,6


In [0]:
scenarios= spark.sql(
    f"""
          select * 
          from produced.drillout_model_scenarios 
          where 1=1 
          {scenario_condition}
          """
).collect()

In [0]:
# Constants declaration
cutoff_perc_for_misc_operators = 5
buffer_days_for_rig_movement = 10
current_date = datetime.now()

# Cycle Time Model

In [0]:
for scenario in scenarios:
    params = {}
    params["basin_of_interest"] = scenario.basin
    params["cutoff_date_for_training_data"] = (
        (
            current_date
            - relativedelta(months=12 * scenario.cycle_times_min_cutoff_years)
        )
        .replace(day=1)
        .date()
    )
    params["flow_unit_of_interest"] = scenario.flow_unit

    params["scenario_id"] = scenario.scenario_id
    params['current_date'] = current_date.date()
    dbutils.notebook.run("./01-CYCLE-TIME-MODEL", 0, params)

# Rigs Model

In [0]:
for scenario in scenarios:
    params = {}
    params["user_input_for_num_of_rigs"] = scenario.number_of_rigs
    params["basin_of_interest"] = scenario.basin
    params["flow_unit_of_interest"] = scenario.flow_unit
    params["desired_active_rig_date"] = (
        (current_date - relativedelta(months=scenario.active_rigs_min_cutoff_months))
        .replace(day=1)
        .date()
    )
    params["cutoff_date_to_train_rig_model"] = (
        (current_date - relativedelta(months=12 * scenario.rigs_min_cutoff_years))
        .replace(day=1)
        .date()
    )
    params["cutoff_perc_for_misc_operators"] = cutoff_perc_for_misc_operators
    params["scenario_id"] = scenario.scenario_id

    params['current_date'] = current_date.date()
    
    dbutils.notebook.run("./02-RIG-MODEL", 0, arguments=params)

# Drilling Scheduling Model

In [0]:
for scenario in scenarios:
    params = {}
    params["flow_unit_of_interest"] = scenario.flow_unit
    params['basin_of_interest'] = scenario.basin
    params["inventory_drilling_end_time"] = (
        (current_date + relativedelta(months=12 * int(scenario.forecast_years)))
        .replace(day=1)
        .date()
    )
    params["buffer_days_for_rig_movement"] = buffer_days_for_rig_movement
    params["scenario_id"] = scenario.scenario_id
 
    dbutils.notebook.run("./03-DRILLING-SCHEDULING-MODEL", 0, params)

# Production Estimation Model

In [0]:
for scenario in scenarios:
    params = {}
    params["basin_of_interest"] = scenario.basin
    params['flowunit_of_interest'] = scenario.flow_unit
    params["desired_minimum_date_for_producing_wells"] = (
        (
            current_date
            - relativedelta(months=12 * scenario.cycle_times_min_cutoff_years)
        )
        .replace(day=1)
        .date()
    )
    params["desired_maximum_date_for_producing_wells"] = (
        (current_date + relativedelta(months=12 * int(scenario.forecast_years)))
        .replace(day=1)
        .date()
    ) 
    params["desired_wip_wells_date"] = (
        (current_date - relativedelta(months=scenario.active_rigs_min_cutoff_months))
        .replace(day=1)
        .date()
    )
    params["cutoff_first_prod_date_for_wip_wells"] = (
        (current_date - relativedelta(months=scenario.active_rigs_min_cutoff_months))
        .replace(day=1)
        .date()
    )
    params["desired_first_prod_date_for_wip_wells"] = (
        (current_date - relativedelta(months=scenario.active_rigs_min_cutoff_months))
        .replace(day=1)
        .date()
    )
    params["buffer_days_for_rig_movement"] = buffer_days_for_rig_movement
    params["cutoff_date_for_new_wells"] = (
        (current_date - relativedelta(months=scenario.active_rigs_min_cutoff_months))
        .replace(day=1)
        .date()
    )
    params['current_date'] = current_date.date() 
    
    params['desired_minimum_firstproddate_for_completed_drilled_wells'] =  (
        (current_date - relativedelta(months=12 * scenario.cycle_times_min_cutoff_years))
        .replace(day=1)
        .date()
    )
    params["scenario_id"] = scenario.scenario_id
    # params["scenario_id"] = 'testing'
    dbutils.notebook.run("./04-PRODUCTION-ESTIMATION-MODEL", 0, params)